In [ ]:
import pandas as pd
import numpy as np
import datetime

## Creating measurement TSV for TJU redacted PHI cohort:

In [ ]:
#TJU demographic tables that include test measurements:
directory1 = '/content/drive/MyDrive/TJU_RSNA_RICORD_COVID_positive_demographics_2021-11-02.csv'
directory2 = '/content/drive/MyDrive/RSNA_MIDRC_TJU_ Covid_Negative_Population_V2_fix_deidentified.csv'

#patients in this submission:
directory3 = '/content/case_RSNA_20221206.csv'

#measurements from previous TJU submissions:
directory4 = '/content/CORRECTED_measurement_RSNA_20220812.csv'
directory5 = '/content/CORRECTED_measurement_RSNA_20220819.csv'

In [ ]:
df1 = pd.read_csv(directory1)
df2 = pd.read_csv(directory2)
df3 = pd.read_csv(directory3)

df4 = pd.read_csv(directory4)
df5 = pd.read_csv(directory5)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Starting with gathering measurements from the TJU positive demographics table. We need to clean up this table so it's useful:

#### First we need to drop the the rows with null values in the patient-id or covid reported date columns:

In [ ]:
df1 = df1.dropna(subset = ['ANON-PatientID', 'ANON-COVID_REPORTED_DATE'])

#### We are interested in these specific columns:

In [ ]:
df1 = df1[['ANON-PatientID', 'ANON-COVID_REPORTED_DATE', 'COVID_ORDER_NAME', 'COVID_STATUS']]

In [ ]:
df1.rename(columns={'ANON-COVID_REPORTED_DATE': 'ANON_COVID_REPORTED_DATE', 'COVID_ORDER_NAME': 'ORDER_NAME', 'COVID_STATUS': 'RESULT_VALUE', 'ANON-PatientID':'case_ids'}, inplace=True)

#### Drop any duplicate measurements:

In [ ]:
df1 = df1.drop_duplicates()
df1

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE
0,514382-005370,10/2/2019,SARS COV-2 (COVID-19) NAA,POSITIVE
1,514382-007424,7/10/2019,SARS COV-2 (COVID-19) EXTERNAL,POSITIVE
2,514382-004492,4/18/2016,SARS COV-2 (COVID-19) + INFLUENZA A/B,POSITIVE
3,514382-011373,2/1/2012,SARS COV-2 (COVID-19) + INFLUENZA A/B,POSITIVE
4,514382-003854,7/11/2018,SARS COV-2 (COVID-19) NAA,POSITIVE
...,...,...,...,...
15660,514382-012584,9/2/2018,SARS COV-2 (COVID-19) + INFLUENZA A/B,POSITIVE
15661,514382-007966,6/2/2013,SARS COV-2 (COVID-19) + INFLUENZA A/B,POSITIVE
15662,514382-006121,6/21/2013,SARS COV-2 (COVID-19) + INFLUENZA A/B,POSITIVE
15664,514382-008526,5/3/2015,SARS COV-2 (COVID-19) EXTERNAL,POSITIVE


#### Converting dates to datetime values:

In [ ]:
df1['ANON_COVID_REPORTED_DATE']= pd.to_datetime(df1['ANON_COVID_REPORTED_DATE'])

#### Sorting values by patient id and reported date:

In [ ]:
df1 = df1.sort_values(by=['case_ids', 'ANON_COVID_REPORTED_DATE'])

In [ ]:
df1

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE
3863,514382-000001,2011-01-05,SARS COV-2 (COVID-19) NAA,POSITIVE
1754,514382-000002,2011-02-05,SARS COV-2 (COVID-19) (INPATIENT),POSITIVE
1760,514382-000003,2012-03-22,SARS COV-2 (COVID-19) NAA,POSITIVE
3284,514382-000004,2017-10-22,SARS COV-2 (COVID-19) (INPATIENT),POSITIVE
10016,514382-000005,2017-11-27,NaN,POSITIVE
...,...,...,...,...
12507,514382-014307,2013-07-21,NaN,POSITIVE
11034,514382-014308,2015-01-29,SARS COV-2 (COVID-19) + INFLUENZA A/B,POSITIVE
14000,514382-014309,2013-11-15,NaN,POSITIVE
5739,514382-014310,2017-11-25,"SARS COV-2 (COVID-19) WITH INFLUENZA A, INFLUE...",POSITIVE


#### Mapping order name values to accepted MIDRC data model values:

In [ ]:
df1['ORDER_NAME'].unique()

array(['SARS COV-2 (COVID-19) NAA', 'SARS COV-2 (COVID-19) (INPATIENT)',
       nan, 'SARS COV-2 (COVID-19) NJ STATE (LAB USE ONLY)',
       'SARS COV-2 (COVID-19) (LABCORP)',
       'TJU SARS COV-2 (LAB ORDER ONLY)',
       'SARS CORONAVIRUS WITH COV-2 RNA, QUAL RT-PCR (QUEST)',
       'SARS COV-2 (COVID-19) EXTERNAL',
       'SARS COV-2 (COVID-19) (EMPLOYEE OR PAT ONLY)',
       'SARS COV-2 (COVID-19) (JEFFERSON LAB)',
       'SARS COV-2 (COVID-19) + INFLUENZA A/B',
       'SARS COV-2 (COVID-19) WITH INFLUENZA A, INFLUENZA B AND RESPIRATORY SYNCYTIAL VIRUS',
       'SARS COV-2 (COVID-19) NAA - LABCORP / QUEST'], dtype=object)

In [ ]:
df1['ORDER_NAME'] = df1['ORDER_NAME'].map({'SARS COV-2 (COVID-19) NAA': 'RT-PCR', 'SARS COV-2 (COVID-19) (INPATIENT)': 'RT-PCR',
                             'SARS COV-2 (COVID-19) NJ STATE (LAB USE ONLY)': 'RT-PCR',
                             'SARS COV-2 (COVID-19) (LABCORP)': 'RT-PCR',
                             'TJU SARS COV-2 (LAB ORDER ONLY)': 'RT-PCR',
                             'SARS CORONAVIRUS WITH COV-2 RNA, QUAL RT-PCR (QUEST)': 'RT-PCR',
                             'SARS COV-2 (COVID-19) EXTERNAL': 'Not Reported',
                             'SARS COV-2 (COVID-19) (EMPLOYEE OR PAT ONLY)': 'RT-PCR',
                             'SARS COV-2 (COVID-19) (JEFFERSON LAB)': 'RT-PCR',
                             'SARS COV-2 (COVID-19) + INFLUENZA A/B': 'RT-PCR',
                             'SARS COV-2 (COVID-19) WITH INFLUENZA A, INFLUENZA B AND RESPIRATORY SYNCYTIAL VIRUS': 'RT-PCR',
                             'SARS COV-2 (COVID-19) NAA - LABCORP / QUEST': 'RT-PCR'})

In [ ]:
df1['ORDER_NAME'].fillna('Not Reported', inplace = True)

In [ ]:
df1['ORDER_NAME'].unique()

array(['RT-PCR', 'Not Reported'], dtype=object)

#### Mapping result values to accepted MIDRC data model values:

In [ ]:
df1['RESULT_VALUE'].unique()

array(['POSITIVE'], dtype=object)

In [ ]:
df1['RESULT_VALUE'] = df1['RESULT_VALUE'].map({'POSITIVE': 'Positive'})

#### The table for positive measurements looks good now:

In [ ]:
df1

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE
3863,514382-000001,2011-01-05,RT-PCR,Positive
1754,514382-000002,2011-02-05,RT-PCR,Positive
1760,514382-000003,2012-03-22,RT-PCR,Positive
3284,514382-000004,2017-10-22,RT-PCR,Positive
10016,514382-000005,2017-11-27,Not Reported,Positive
...,...,...,...,...
12507,514382-014307,2013-07-21,Not Reported,Positive
11034,514382-014308,2015-01-29,RT-PCR,Positive
14000,514382-014309,2013-11-15,Not Reported,Positive
5739,514382-014310,2017-11-25,RT-PCR,Positive


### Onto the TJU negative demographics table. Similarly, we need to also clean up this table so it's useful:

In [ ]:
df2

,ANON-PatientID,Age,Gender Desc,ZIP,Race Desc,Ethnicity Desc,ANON_ENC_TIMESTAMP,ANON_ADMISSION_TIMESTAMP,ORDER_NAME,ANON_ORDER_TIMESTAMP,RESULT_VALUE,ANON_COVID_REPORTED_DATE,HOSP_LOS,HOSP_ENCTR_IND,ICU_ENCTR_IND,ICU_DAYS_TOTAL,VENT_YN,VENT_DAYS_TOTAL,DISCHARGE_DISPOSITION,MORTALITY_FLAG
0,514382-000002,82,Male,191,asian,"Not Hispanic, Latino/a, or Spanish origin",1/5/2012,1/5/2012,SARS COV-2 (COVID-19) NAA,1/5/2012,Negative,1/5/2012,1.0,Y,N,NaN,N,NaN,DISCHARGED TO HOME OR SELF CARE (ROUTINE DISCH...,No
1,514382-000002,82,Male,191,asian,"Not Hispanic, Latino/a, or Spanish origin",5/29/2011,5/29/2011,SARS COV-2 (COVID-19) NAA,5/29/2011,Negative,5/30/2011,1.0,Y,N,NaN,N,NaN,DISCHARGED TO HOME OR SELF CARE (ROUTINE DISCH...,No
2,514382-000002,82,Male,191,asian,"Not Hispanic, Latino/a, or Spanish origin",6/8/2011,6/8/2011,SARS COV-2 (COVID-19) NAA,6/8/2011,Negative,6/8/2011,5.0,Y,N,NaN,N,NaN,DISCHARGED/TRANSFERRED TO HOME UNDER CARE OF O...,No
3,514382-000014,54,Male,80,black or african american,"Not Hispanic, Latino/a, or Spanish origin",7/7/2018,NaN,SARS COV-2 (COVID-19) (LABCORP),7/7/2018,Not Detected,7/10/2018,NaN,N,N,NaN,N,NaN,NaN,NaN
4,514382-000015,79,Female,80,asian,"Not Hispanic, Latino/a, or Spanish origin",3/2/2012,3/2/2012,SARS COV-2 (COVID-19) NAA,3/4/2012,Negative,3/5/2012,7.0,Y,N,NaN,N,NaN,DISCHARGED/TRANSFERRED TO SKILLED NURSING FACI...,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70217,514382-063591,57,Male,191.0,black or african american,"Not Hispanic, Latino/a, or Spanish origin",3/10/2020,3/10/2020,SARS COV-2 (COVID-19) NAA,3/24/2020,Negative,3/24/2020,18.0,Y,Y,3.16,N,NaN,DISCHARGED/TRANSFERRED TO HOME UNDER CARE OF O...,No
70218,514382-063592,87,Male,191.0,white or caucasian,"Not Hispanic, Latino/a, or Spanish origin",9/5/2014,9/5/2014,SARS COV-2 (COVID-19) NAA,9/10/2014,Negative,9/10/2014,5.0,Y,N,NaN,N,NaN,DISCHARGED/TRANSFERRED TO SKILLED NURSING FACI...,No
70219,514382-063593,84,Male,191.0,hispanic,Puerto Rican,1/8/2015,1/8/2015,SARS COV-2 (COVID-19) NAA,1/9/2015,Negative,1/12/2015,4.0,Y,N,NaN,N,NaN,DISCHARGED/TRANSFERRED TO HOME UNDER CARE OF O...,No
70220,514382-063594,56,Female,191.0,white or caucasian,"Not Hispanic, Latino/a, or Spanish origin",5/17/2018,5/17/2018,SARS COV-2 (COVID-19) EXTERNAL,5/19/2018,Negative,5/17/2018,6.0,Y,N,NaN,N,NaN,DISCHARGED TO HOME OR SELF CARE (ROUTINE DISCH...,No


In [ ]:
df2 = df2.dropna(subset = ['ANON-PatientID', 'ANON_COVID_REPORTED_DATE'])

In [ ]:
df2 = df2[['ANON-PatientID', 'ANON_COVID_REPORTED_DATE', 'ORDER_NAME', 'RESULT_VALUE']]

In [ ]:
df2 = df2.drop_duplicates()
df2

,ANON-PatientID,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE
0,514382-000002,1/5/2012,SARS COV-2 (COVID-19) NAA,Negative
1,514382-000002,5/30/2011,SARS COV-2 (COVID-19) NAA,Negative
2,514382-000002,6/8/2011,SARS COV-2 (COVID-19) NAA,Negative
3,514382-000014,7/10/2018,SARS COV-2 (COVID-19) (LABCORP),Not Detected
4,514382-000015,3/5/2012,SARS COV-2 (COVID-19) NAA,Negative
...,...,...,...,...
70217,514382-063591,3/24/2020,SARS COV-2 (COVID-19) NAA,Negative
70218,514382-063592,9/10/2014,SARS COV-2 (COVID-19) NAA,Negative
70219,514382-063593,1/12/2015,SARS COV-2 (COVID-19) NAA,Negative
70220,514382-063594,5/17/2018,SARS COV-2 (COVID-19) EXTERNAL,Negative


In [ ]:
dict_rename = {'ANON-PatientID':'case_ids'}
df2.rename(columns=dict_rename,
          inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
df2['ANON_COVID_REPORTED_DATE']= pd.to_datetime(df2['ANON_COVID_REPORTED_DATE'])

In [ ]:
df2 = df2.sort_values(by=['case_ids', 'ANON_COVID_REPORTED_DATE'])

In [ ]:
df2

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE
1,514382-000002,2011-05-30,SARS COV-2 (COVID-19) NAA,Negative
2,514382-000002,2011-06-08,SARS COV-2 (COVID-19) NAA,Negative
0,514382-000002,2012-01-05,SARS COV-2 (COVID-19) NAA,Negative
3,514382-000014,2018-07-10,SARS COV-2 (COVID-19) (LABCORP),Not Detected
4,514382-000015,2012-03-05,SARS COV-2 (COVID-19) NAA,Negative
...,...,...,...,...
70217,514382-063591,2020-03-24,SARS COV-2 (COVID-19) NAA,Negative
70218,514382-063592,2014-09-10,SARS COV-2 (COVID-19) NAA,Negative
70219,514382-063593,2015-01-12,SARS COV-2 (COVID-19) NAA,Negative
70220,514382-063594,2018-05-17,SARS COV-2 (COVID-19) EXTERNAL,Negative


In [ ]:
df2['ORDER_NAME'].unique()

array(['SARS COV-2 (COVID-19) NAA', 'SARS COV-2 (COVID-19) (LABCORP)',
       'SARS COV-2 (COVID-19) EXTERNAL',
       'SARS COV-2 (COVID-19) (JEFFERSON LAB)',
       'SARS COV-2 (COVID-19) NJ STATE (LAB USE ONLY)',
       'SARS COV-2 (LAB ORDER ONLY)',
       'SARS CORONAVIRUS WITH COV-2 RNA, QUAL RT-PCR (QUEST)'],
      dtype=object)

In [ ]:
df2['ORDER_NAME'] = df2['ORDER_NAME'].map({'SARS COV-2 (COVID-19) NAA': 'RT-PCR',
                                                       'SARS COV-2 (COVID-19) (LABCORP)': 'RT-PCR',
                                                       'SARS COV-2 (COVID-19) EXTERNAL': 'Not Reported',
                                                       'SARS COV-2 (COVID-19) (JEFFERSON LAB)': 'RT-PCR',
                                                       'SARS COV-2 (COVID-19) NJ STATE (LAB USE ONLY)': 'RT-PCR',
                                                       'SARS COV-2 (LAB ORDER ONLY)': 'RT-PCR',
                                                       'SARS CORONAVIRUS WITH COV-2 RNA, QUAL RT-PCR (QUEST)': 'RT-PCR'})

In [ ]:
df2['ORDER_NAME'].fillna('Not Reported', inplace = True)

In [ ]:
df2['ORDER_NAME'].unique()

array(['RT-PCR', 'Not Reported'], dtype=object)

In [ ]:
df2['RESULT_VALUE'].unique()

array(['Negative', 'Not Detected', 'Negative ', 'NOT DETECTED', 'ik'],
      dtype=object)

In [ ]:
df2['RESULT_VALUE'] = df2['RESULT_VALUE'].map({'Negative ': 'Negative', 'Not Detected': 'Negative',
                                               'NOT DETECTED': 'Negative', 'ik': 'Not Reported',
                                               'Negative': 'Negative'})

In [ ]:
df2['RESULT_VALUE'].unique()

array(['Negative', 'Not Reported'], dtype=object)

#### The neagtive measurements table is now ready:

In [ ]:
df2

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE
1,514382-000002,2011-05-30,RT-PCR,Negative
2,514382-000002,2011-06-08,RT-PCR,Negative
0,514382-000002,2012-01-05,RT-PCR,Negative
3,514382-000014,2018-07-10,RT-PCR,Negative
4,514382-000015,2012-03-05,RT-PCR,Negative
...,...,...,...,...
70217,514382-063591,2020-03-24,RT-PCR,Negative
70218,514382-063592,2014-09-10,RT-PCR,Negative
70219,514382-063593,2015-01-12,RT-PCR,Negative
70220,514382-063594,2018-05-17,Not Reported,Negative


### Appending both measurement tables (positive and negative) to gather all measurements:

In [ ]:
all_measurements = df1.append(df2)

In [ ]:
all_measurements

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE
3863,514382-000001,2011-01-05,RT-PCR,Positive
1754,514382-000002,2011-02-05,RT-PCR,Positive
1760,514382-000003,2012-03-22,RT-PCR,Positive
3284,514382-000004,2017-10-22,RT-PCR,Positive
10016,514382-000005,2017-11-27,Not Reported,Positive
...,...,...,...,...
70217,514382-063591,2020-03-24,RT-PCR,Negative
70218,514382-063592,2014-09-10,RT-PCR,Negative
70219,514382-063593,2015-01-12,RT-PCR,Negative
70220,514382-063594,2018-05-17,Not Reported,Negative


### We will use the case node table, df3, to gather the measurements from all the patients in the cohort:

In [ ]:
df3

,type,submitter_id,site_id,case_ids,age_at_index_gt89,age_at_index,sex,race,ethnicity,gen3_linked_subjects_available,zip,index_event,linked_external_data,covid19_positive,icu_indicator,ventilator_indicator,country_of_origin,token_record_id,treatment_info,datasets
0,NaN,514382-000004,514382,514382-000004,No,68.0,Male,White,Not Hispanic or Latino,0,80,COVID-19 Test,NaN,Yes,True,False,NaN,NaN,NaN,NaN
1,NaN,514382-000005,514382,514382-000005,No,80.0,Male,Asian,Not Hispanic or Latino,0,80,COVID-19 Test,NaN,Yes,True,False,NaN,NaN,NaN,NaN
2,NaN,514382-000007,514382,514382-000007,No,63.0,Female,Black or African American,Not Hispanic or Latino,0,81,COVID-19 Test,NaN,Yes,True,True,NaN,NaN,NaN,NaN
3,NaN,514382-000008,514382,514382-000008,No,85.0,Male,Asian,Not Hispanic or Latino,0,80,COVID-19 Test,NaN,Yes,False,False,NaN,NaN,NaN,NaN
4,NaN,514382-000015,514382,514382-000015,No,79.0,Female,Asian,Not Hispanic or Latino,0,80,COVID-19 Test,NaN,No,False,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,NaN,514382-041489,514382,514382-041489,No,26.0,Male,White,Not Hispanic or Latino,0,81,COVID-19 Test,NaN,No,False,False,NaN,NaN,NaN,NaN
2861,NaN,514382-041517,514382,514382-041517,No,76.0,Female,White,Not Hispanic or Latino,0,81,COVID-19 Test,NaN,No,True,True,NaN,NaN,NaN,NaN
2862,NaN,514382-041532,514382,514382-041532,No,70.0,Male,White,Not Reported,0,80,COVID-19 Test,NaN,No,False,False,NaN,NaN,NaN,NaN
2863,NaN,514382-041548,514382,514382-041548,No,50.0,Male,Black or African American,Not Hispanic or Latino,0,80,COVID-19 Test,NaN,No,False,False,NaN,NaN,NaN,NaN


#### Gather the subset of measurements that belong only to the patients in this batch:

In [ ]:
sub_measurements = all_measurements.loc[all_measurements.case_ids.isin(df3['case_ids'])]

In [ ]:
sub_measurements

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE
3284,514382-000004,2017-10-22,RT-PCR,Positive
10016,514382-000005,2017-11-27,Not Reported,Positive
4201,514382-000007,2018-06-01,RT-PCR,Positive
5001,514382-000008,2010-06-03,RT-PCR,Positive
9765,514382-000015,2011-04-09,RT-PCR,Positive
...,...,...,...,...
45287,514382-041517,2019-08-30,RT-PCR,Negative
45310,514382-041532,2020-10-03,RT-PCR,Negative
45329,514382-041548,2014-07-27,RT-PCR,Negative
45330,514382-041549,2015-09-20,RT-PCR,Negative


#### Sanity check: Make sure that there exists at least one measurement for each patient in the cohort:

In [ ]:
sub_measurements.drop_duplicates(subset = 'case_ids')

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE
3284,514382-000004,2017-10-22,RT-PCR,Positive
10016,514382-000005,2017-11-27,Not Reported,Positive
4201,514382-000007,2018-06-01,RT-PCR,Positive
5001,514382-000008,2010-06-03,RT-PCR,Positive
9765,514382-000015,2011-04-09,RT-PCR,Positive
...,...,...,...,...
45245,514382-041489,2013-12-16,RT-PCR,Negative
45287,514382-041517,2019-08-30,RT-PCR,Negative
45310,514382-041532,2020-10-03,RT-PCR,Negative
45329,514382-041548,2014-07-27,RT-PCR,Negative


#### Make sure that the dates are all converted to datetime:

In [ ]:
sub_measurements['ANON_COVID_REPORTED_DATE']= pd.to_datetime(sub_measurements['ANON_COVID_REPORTED_DATE'])

<ipython-input-40-3e08af4189e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_measurements['ANON_COVID_REPORTED_DATE']= pd.to_datetime(sub_measurements['ANON_COVID_REPORTED_DATE'])


#### Sort all rows again by patient id and covid reported date:

In [ ]:
sub_measurements = sub_measurements.sort_values(by=['case_ids', 'ANON_COVID_REPORTED_DATE'])

In [ ]:
sub_measurements

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE
3284,514382-000004,2017-10-22,RT-PCR,Positive
10016,514382-000005,2017-11-27,Not Reported,Positive
4201,514382-000007,2018-06-01,RT-PCR,Positive
5001,514382-000008,2010-06-03,RT-PCR,Positive
9765,514382-000015,2011-04-09,RT-PCR,Positive
...,...,...,...,...
45287,514382-041517,2019-08-30,RT-PCR,Negative
45310,514382-041532,2020-10-03,RT-PCR,Negative
45329,514382-041548,2014-07-27,RT-PCR,Negative
45330,514382-041549,2015-09-20,RT-PCR,Negative


#### Create submitter_ids in the format Measurement_patient-id_0measurement-count:

In [ ]:
mrns = sub_measurements['case_ids'].tolist()
print(mrns)

['514382-000004', '514382-000005', '514382-000007', '514382-000008', '514382-000015', '514382-000015', '514382-000018', '514382-000021', '514382-000035', '514382-000043', '514382-000044', '514382-000051', '514382-000057', '514382-000057', '514382-000064', '514382-000067', '514382-000079', '514382-000083', '514382-000087', '514382-000089', '514382-000091', '514382-000099', '514382-000101', '514382-000114', '514382-000121', '514382-000124', '514382-000132', '514382-000133', '514382-000138', '514382-000138', '514382-000138', '514382-000155', '514382-000156', '514382-000164', '514382-000165', '514382-000171', '514382-000171', '514382-000179', '514382-000179', '514382-000188', '514382-000189', '514382-000193', '514382-000198', '514382-000201', '514382-000211', '514382-000221', '514382-000225', '514382-000247', '514382-000250', '514382-000254', '514382-000258', '514382-000261', '514382-000268', '514382-000270', '514382-000271', '514382-000271', '514382-000271', '514382-000274', '514382-00028

In [ ]:
count = 1
for i in range(len(mrns)):
  if i !=(len(mrns)-1):
    next = mrns[i+1]
  else:
    next = 'end'
  current = mrns[i]
  mrns[i] = mrns[i] + "_0" + str(count)
  if current == next:
    count = count+1
  else:
    count = 1

In [ ]:
sub_measurements = sub_measurements.assign(submitter_id=mrns)
sub_measurements

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE,submitter_id
3284,514382-000004,2017-10-22,RT-PCR,Positive,514382-000004_01
10016,514382-000005,2017-11-27,Not Reported,Positive,514382-000005_01
4201,514382-000007,2018-06-01,RT-PCR,Positive,514382-000007_01
5001,514382-000008,2010-06-03,RT-PCR,Positive,514382-000008_01
9765,514382-000015,2011-04-09,RT-PCR,Positive,514382-000015_01
...,...,...,...,...,...
45287,514382-041517,2019-08-30,RT-PCR,Negative,514382-041517_01
45310,514382-041532,2020-10-03,RT-PCR,Negative,514382-041532_01
45329,514382-041548,2014-07-27,RT-PCR,Negative,514382-041548_01
45330,514382-041549,2015-09-20,RT-PCR,Negative,514382-041549_01


In [ ]:
sub_measurements["submitter_id"] =   "Measurement_" + sub_measurements["submitter_id"]

#### Calculate test_days_from_index:

In [ ]:
sub_measurements = sub_measurements.reset_index(drop=True)

In [ ]:
test_days_from_index = []
track = ''
date_time = datetime.date.today()
for ind in sub_measurements.index:
    if sub_measurements['case_ids'][ind] == track:
      test_days_from_index.append((sub_measurements['ANON_COVID_REPORTED_DATE'][ind]-date_time).days)
    else:
      test_days_from_index.append(0)
      date_time = sub_measurements['ANON_COVID_REPORTED_DATE'][ind]
    track = sub_measurements['case_ids'][ind]

In [ ]:
sub_measurements['test_days_from_index'] = test_days_from_index

In [ ]:
sub_measurements

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE,submitter_id,test_days_from_index
0,514382-000004,2017-10-22,RT-PCR,Positive,Measurement_514382-000004_01,0
1,514382-000005,2017-11-27,Not Reported,Positive,Measurement_514382-000005_01,0
2,514382-000007,2018-06-01,RT-PCR,Positive,Measurement_514382-000007_01,0
3,514382-000008,2010-06-03,RT-PCR,Positive,Measurement_514382-000008_01,0
4,514382-000015,2011-04-09,RT-PCR,Positive,Measurement_514382-000015_01,0
...,...,...,...,...,...,...
3872,514382-041517,2019-08-30,RT-PCR,Negative,Measurement_514382-041517_01,0
3873,514382-041532,2020-10-03,RT-PCR,Negative,Measurement_514382-041532_01,0
3874,514382-041548,2014-07-27,RT-PCR,Negative,Measurement_514382-041548_01,0
3875,514382-041549,2015-09-20,RT-PCR,Negative,Measurement_514382-041549_01,0


#### Keep the measurements that weren't submitted in the previous two submissions of TJU data:

In [ ]:
sub_measurements_left = sub_measurements.loc[~sub_measurements.case_ids.isin(df4['case_ids'])]

In [ ]:
sub_measurements_left = sub_measurements_left.loc[~sub_measurements_left.case_ids.isin(df5['case_ids'])]
sub_measurements_left

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE,submitter_id,test_days_from_index
0,514382-000004,2017-10-22,RT-PCR,Positive,Measurement_514382-000004_01,0
1,514382-000005,2017-11-27,Not Reported,Positive,Measurement_514382-000005_01,0
3,514382-000008,2010-06-03,RT-PCR,Positive,Measurement_514382-000008_01,0
4,514382-000015,2011-04-09,RT-PCR,Positive,Measurement_514382-000015_01,0
5,514382-000015,2012-03-05,RT-PCR,Negative,Measurement_514382-000015_02,331
...,...,...,...,...,...,...
3872,514382-041517,2019-08-30,RT-PCR,Negative,Measurement_514382-041517_01,0
3873,514382-041532,2020-10-03,RT-PCR,Negative,Measurement_514382-041532_01,0
3874,514382-041548,2014-07-27,RT-PCR,Negative,Measurement_514382-041548_01,0
3875,514382-041549,2015-09-20,RT-PCR,Negative,Measurement_514382-041549_01,0


#### We're left with 2896 measurements total. Let's make sure that the submitter_ids are unique:

In [ ]:
sub_measurements_left.drop_duplicates(subset = 'submitter_id')

,case_ids,ANON_COVID_REPORTED_DATE,ORDER_NAME,RESULT_VALUE,submitter_id,test_days_from_index
0,514382-000004,2017-10-22,RT-PCR,Positive,Measurement_514382-000004_01,0
1,514382-000005,2017-11-27,Not Reported,Positive,Measurement_514382-000005_01,0
3,514382-000008,2010-06-03,RT-PCR,Positive,Measurement_514382-000008_01,0
4,514382-000015,2011-04-09,RT-PCR,Positive,Measurement_514382-000015_01,0
5,514382-000015,2012-03-05,RT-PCR,Negative,Measurement_514382-000015_02,331
...,...,...,...,...,...,...
3872,514382-041517,2019-08-30,RT-PCR,Negative,Measurement_514382-041517_01,0
3873,514382-041532,2020-10-03,RT-PCR,Negative,Measurement_514382-041532_01,0
3874,514382-041548,2014-07-27,RT-PCR,Negative,Measurement_514382-041548_01,0
3875,514382-041549,2015-09-20,RT-PCR,Negative,Measurement_514382-041549_01,0


#### Rename the columns to adhere to the MIDRC data model:

In [ ]:
sub_measurements_left['test_name'] = 'COVID-19'


In [ ]:
sub_measurements_left['cases'] = sub_measurements_left['case_ids']

In [ ]:
dict_rename = {'RESULT_VALUE':'test_result_text',
               'ORDER_NAME': 'test_method'}
sub_measurements_left.rename(columns=dict_rename,
          inplace=True)

In [ ]:
sub_measurements_left

,case_ids,ANON_COVID_REPORTED_DATE,test_method,test_result_text,submitter_id,test_days_from_index,test_name,cases
0,514382-000004,2017-10-22,RT-PCR,Positive,Measurement_514382-000004_01,0,COVID-19,514382-000004
1,514382-000005,2017-11-27,Not Reported,Positive,Measurement_514382-000005_01,0,COVID-19,514382-000005
3,514382-000008,2010-06-03,RT-PCR,Positive,Measurement_514382-000008_01,0,COVID-19,514382-000008
4,514382-000015,2011-04-09,RT-PCR,Positive,Measurement_514382-000015_01,0,COVID-19,514382-000015
5,514382-000015,2012-03-05,RT-PCR,Negative,Measurement_514382-000015_02,331,COVID-19,514382-000015
...,...,...,...,...,...,...,...,...
3872,514382-041517,2019-08-30,RT-PCR,Negative,Measurement_514382-041517_01,0,COVID-19,514382-041517
3873,514382-041532,2020-10-03,RT-PCR,Negative,Measurement_514382-041532_01,0,COVID-19,514382-041532
3874,514382-041548,2014-07-27,RT-PCR,Negative,Measurement_514382-041548_01,0,COVID-19,514382-041548
3875,514382-041549,2015-09-20,RT-PCR,Negative,Measurement_514382-041549_01,0,COVID-19,514382-041549


#### Save file:

In [ ]:
sub_measurements_left.to_csv('measurement_RSNA_20221206.csv', index = False)